In [41]:
import torch
from torchvision.datasets import CIFAR10, CIFAR100, STL10, ImageFolder
from augmentations import SimCLRTransform, SimSiamTransform

from torch.utils.data import DataLoader


In [42]:
torch.cuda.is_available()

True

In [43]:
import os
import re

regex = re.compile('Loss: ([0-9]*.[0-9]*)')
regex2 = re.compile('Epoch ([0-9]*)')

location = "/Users/Luctopikut/bep/BEP-DoGo/data/output/luc/"
# location = './data/output/luc'
file_name ='train_cifar10.log'

def get_loss(location, file_name):
    result = dict()
    counter =0
    for filename in os.listdir(location):
        if filename == file_name:
            with open(os.path.join(location, file_name), "r") as file:
                
                for line in file:
                    if 'Epoch [' in line:
                        if 'Loss:' in line:
                            counter +=1
                            result[counter] = regex.findall(line)[0]
    return result

results = get_loss(location, file_name)

In [44]:
results

{1: '0.8085398039814219',
 2: '0.6031583615814347',
 3: '0.57363679892866',
 4: '0.557368598541761',
 5: '0.5470239107598023',
 6: '0.5384301504636385',
 7: '0.5313324332686074',
 8: '0.5258952523301165',
 9: '0.5216031613120113',
 10: '0.5169989917741482',
 11: '0.5130166432494858',
 12: '0.5096689833619867',
 13: '0.5058405970234469',
 14: '0.5031225805630885',
 15: '0.5004946865500456',
 16: '0.4986368755933391',
 17: '0.49505706995456333',
 18: '0.49274758590632173',
 19: '0.49128850688596803',
 20: '0.4893899194626923',
 21: '0.48709229790571945',
 22: '0.48565842507474394',
 23: '0.483964079901217',
 24: '0.4819037293544017',
 25: '0.48047962829650165',
 26: '0.4791675073330302',
 27: '0.4780332723250949',
 28: '0.4761667682015034',
 29: '0.4742799716586449',
 30: '0.47349622493318044',
 31: '0.47236064363674946',
 32: '0.4709144782005663',
 33: '0.4704091947480857',
 34: '0.4688941843538399',
 35: '0.4681346240756382',
 36: '0.46697469098291483',
 37: '0.46612209925450476',
 38:

In [45]:
import matplotlib.pyplot as plt

# x, y = zip(*results.items()) # unpack a list of pairs into two tuples


# plt.plot(x,y.astype(int))
# plt.show()

In [46]:
import pandas as pd
df = pd.DataFrame(results.items())

In [47]:
df.rename(columns={0:'x', 1:'y'}, inplace=True)


In [48]:
df.columns

Index(['x', 'y'], dtype='object')

In [49]:
df['y'] = pd.to_numeric(df['y'])
# 

In [50]:
# [tensor(50., device='cuda:0'), tensor(33.3333, device='cuda:0'), tensor(50., device='cuda:0'), tensor(16.6667, device='cuda:0'), tensor(0., device='cuda:0'), tensor(42.8571, device='cuda:0'), tensor(72.7273, device='cuda:0'), tensor(14.2857, device='cuda:0'), tensor(85.7143, device='cuda:0'), tensor(88.8889, device='cuda:0')]

In [51]:
# plt.plot(y, data=df)

In [52]:
import plotly.express as px

In [53]:
fig = px.line(df, x="x", y="y", title='Life expectancy in Canada')
fig.show()


In [54]:
transform = SimCLRTransform(32)

train_dataset_no_transform = CIFAR10('/data/input/datasets/CIFAR-10', train=True, download=True)#, transform=transform)
train_dataset = CIFAR10('/data/input/datasets/CIFAR-10', train=True, download=True, transform=transform)


Files already downloaded and verified
Files already downloaded and verified


In [55]:
train_loader = DataLoader(train_dataset, batch_size=256, shuffle=False, drop_last=True, num_workers= 4)


In [56]:
from PIL import Image

train_dataset_no_transform

Dataset CIFAR10
    Number of datapoints: 50000
    Root location: /data/input/datasets/CIFAR-10
    Split: Train

In [57]:
label_to_word = {
    0: "Airplane",
    1: "Autombile",
    2: "Bird",
    3: "Cat",
    4: "Deer",
    5: "Dog",
    6: "Frog",
    7: "Horse",
    8: "Ship",
    9: "Truck"
}


In [58]:
# import numpy as np
# for i, ((x1, y1), targets) in enumerate(train_loader):
#     # print(targets)
#     # print(x1[0:10].shape)#, y1)
#     # print(targets[0:10].shape)  
#     # print(y1[0:10].shape)
#     images = x1[0:15]
#     labels = targets[0:15]
#     fig, axs = plt.subplots(1, 1)
#     for i, ax in enumerate(np.reshape(axs, [-1])):

#         ax.imshow(images[0].permute(1, 2, 0))
#     # print(images[0].shape)
#     # plot_15_img(label_to_word, images, labels)
#     break

In [59]:
def multi_acc(pred, label):
  accs_per_label_pct = []
  tags = torch.argmax(pred, dim=1)
  for c in range(3):  # the three classes
    of_c = label == c
    num_total_per_label = of_c.sum()
    of_c &= tags == label
    num_corrects_per_label = of_c.sum()
    accs_per_label_pct.append(num_corrects_per_label / num_total_per_label * 100)
  return accs_per_label_pct

In [78]:
from util.utils import save_checkpoint, log

from datetime import datetime

def train_one_epoch(train_loader, model, criteria, optimizer, scheduler):
    """
    Train one epoch of SSL model
    """
    loss_per_criterion = {}
    total_loss = 0
    for i, ((x1, y1), targets) in enumerate(train_loader):
        x1 = x1.cuda(device=torch.device("cuda"))
        y1 = y1.cuda(device=torch.device("cuda"))
        optimizer.zero_grad()
        print(targets)
        break

        _, _, zx, zy = model(x1, y1)


        # nb_classes = 9

        # confusion_matrix = torch.zeros(nb_classes, nb_classes)
        # with torch.no_grad():
        #     for i, (inputs, classes) in enumerate(dataloaders['val']):
        #         inputs = inputs.to(args.device)
        #         classes = classes.to(args.device)
        #         outputs = model(inputs)
        #         _, preds = torch.max(outputs, 1)
        #         for t, p in zip(classes.view(-1), preds.view(-1)):
        #                 confusion_matrix[t.long(), p.long()] += 1

        # Multiple loss aggregation
    
        loss = torch.tensor(0).to(torch.device("cuda"))
        for k in criteria:
            if k == 'ntxent':
                criterion_loss = criteria[k][0](zx, zy)
            elif k == 'simsiam':
                criterion_loss = criteria[k][0](zx, zy, px, py)
            if k not in loss_per_criterion:
                loss_per_criterion[k] = criterion_loss
            else:
                loss_per_criterion[k] += criterion_loss
            loss = torch.add(loss, torch.mul(criterion_loss, criteria[k][1]))
        print(loss)
        loss.backward()
        # print(loss)
        # print(targets)
        optimizer.step()
        if scheduler is not None:
            scheduler.step()
        if i % 50 == 0:
            log("Batch {}/{}. Loss: {}.  Time elapsed: {} ".format(i, len(train_loader), loss.item(),
                                                                   datetime.now() - datetime.now()))
        total_loss += loss.item()
    return total_loss, loss_per_criterion

In [79]:
import torch.nn as nn
import torchvision
import torch
import torch.nn.functional as F
from models.helper import get_encoder


class SimCLR(nn.Module):
    def __init__(self, img_size, backbone='resnet50'):
        super(SimCLR, self).__init__()
        self.f, projection_size = get_encoder(backbone, img_size)
        if img_size >= 100:
            projection_size = self.f.fc.out_features

        # projection head
        self.g = nn.Sequential(
                                nn.Linear(projection_size, 512, bias=False),
                                nn.BatchNorm1d(512),
                                nn.ReLU(inplace=True),
                                nn.Linear(512, 128, bias=True)
                               )

    def forward(self, x, y=None):
        x = self.f(x)
        feat_x = torch.flatten(x, start_dim=1)
        out_x = self.g(feat_x)

        if y is not None:
            y = self.f(y)
            feat_y = torch.flatten(y, start_dim=1)
            out_y = self.g(feat_y)
            return F.normalize(feat_x, dim=-1), F.normalize(feat_y, dim=-1), F.normalize(out_x, dim=-1),  F.normalize(out_y, dim=-1)
        else:
            return F.normalize(feat_x, dim=-1), F.normalize(out_x, dim=-1)




In [80]:
import torch
import torch.nn as nn
from util.utils import positive_mask
from torch.optim import Adam, SGD


class NTXent(nn.Module):
    """
    The Normalized Temperature-scaled Cross Entropy Loss
    Source: https://github.com/Spijkervet/SimCLR
    """

    def __init__(self):
        super(NTXent, self).__init__()
        self.batch_size = 256
        self.temperature = 0.5
        self.device = torch.device('cuda')
        self.mask = positive_mask(256)
        self.criterion = nn.CrossEntropyLoss(reduction="sum")
        self.similarity_f = nn.CosineSimilarity(dim=2)
        self.N = 2 * self.batch_size

    def forward(self, zx, zy):
        """
        zx: projection output of batch zx
        zy: projection output of batch zy
        :return: normalized loss
        """
        positive_samples, negative_samples = self.sample_no_dict(zx, zy)
        labels = torch.zeros(self.N).to(positive_samples.device).long()
        logits = torch.cat((positive_samples, negative_samples), dim=1)
        loss = self.criterion(logits, labels)
        loss /= self.N
        return loss

    def sample_no_dict(self, zx, zy):
        """
        Positive and Negative sampling without dictionary
        """
        z = torch.cat((zx, zy), dim=0)
        sim = self.similarity_f(z.unsqueeze(1), z.unsqueeze(0)) / self.temperature
        # Since projections are already normalized using F.normalize,
        # below function can be used instead of CosineSimilarity
        # sim = torch.div(torch.matmul(z, z.T), self.temperature)

        # Extract positive samples
        sim_xy = torch.diag(sim, self.batch_size)
        sim_yx = torch.diag(sim, -self.batch_size)
        positive_samples = torch.cat((sim_xy, sim_yx), dim=0).reshape(self.N, 1)

        # Extract negative samples
        negative_samples = sim[self.mask].reshape(self.N, -1)
        return positive_samples, negative_samples



In [81]:
def get_criteria():
    """
    Loss criterion / criteria selection for training
    """
    criteria = {
        'ntxent': [NTXent(), [1, 0][0]]
    }

    return criteria

In [82]:
from torch.optim.lr_scheduler import CosineAnnealingLR

criterion = get_criteria()

model = SimCLR(32, backbone='resnet18')

optimizer = Adam(model.parameters(), lr=0.0003, weight_decay=1.e-6)
scheduler = CosineAnnealingLR(optimizer, T_max=200, eta_min=3e-4)




model = model.to(torch.device("cuda"))
train_one_epoch(train_loader, model, criterion, optimizer, scheduler)

c:\Users\Luctopikut\bep\BEP-DoGo\util\utils.py:36: UserWarning:

masked_fill_ received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\TensorAdvancedIndexing.cpp:1773.)

c:\Users\Luctopikut\anaconda3\envs\main\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning:

The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.

c:\Users\Luctopikut\anaconda3\envs\main\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning:

Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.



tensor([6, 9, 9, 4, 1, 1, 2, 7, 8, 3, 4, 7, 7, 2, 9, 9, 9, 3, 2, 6, 4, 3, 6, 6,
        2, 6, 3, 5, 4, 0, 0, 9, 1, 3, 4, 0, 3, 7, 3, 3, 5, 2, 2, 7, 1, 1, 1, 2,
        2, 0, 9, 5, 7, 9, 2, 2, 5, 2, 4, 3, 1, 1, 8, 2, 1, 1, 4, 9, 7, 8, 5, 9,
        6, 7, 3, 1, 9, 0, 3, 1, 3, 5, 4, 5, 7, 7, 4, 7, 9, 4, 2, 3, 8, 0, 1, 6,
        1, 1, 4, 1, 8, 3, 9, 6, 6, 1, 8, 5, 2, 9, 9, 8, 1, 7, 7, 0, 0, 6, 9, 1,
        2, 2, 9, 2, 6, 6, 1, 9, 5, 0, 4, 7, 6, 7, 1, 8, 1, 1, 2, 8, 1, 3, 3, 6,
        2, 4, 9, 9, 5, 4, 3, 6, 7, 4, 6, 8, 5, 5, 4, 3, 1, 8, 4, 7, 6, 0, 9, 5,
        1, 3, 8, 2, 7, 5, 3, 4, 1, 5, 7, 0, 4, 7, 5, 5, 1, 0, 9, 6, 9, 0, 8, 7,
        8, 8, 2, 5, 2, 3, 5, 0, 6, 1, 9, 3, 6, 9, 1, 3, 9, 6, 6, 7, 1, 0, 9, 5,
        8, 5, 2, 9, 0, 8, 8, 0, 6, 9, 1, 1, 6, 3, 7, 6, 6, 0, 6, 6, 1, 7, 1, 5,
        8, 3, 6, 6, 8, 6, 8, 4, 6, 6, 1, 3, 8, 3, 4, 1])


(0, {})

In [65]:
tenso_list = [torch.randn(3,4), torch.randn(2,5), torch.randn(6,8)]
comb_length =0
for tensor in tenso_list:
    comb_length += tensor.numel()

print(comb_length)

70


In [66]:
tenso_list

[tensor([[-0.0881, -1.3074,  2.4725, -1.8907],
         [-1.5868, -0.7737,  1.8130,  0.8007],
         [-1.6190,  1.0749,  0.4468,  0.5375]]),
 tensor([[ 0.7416, -0.0590, -1.6293,  0.6872,  0.5081],
         [ 0.1445,  0.7972,  0.2143, -0.6485,  0.2030]]),
 tensor([[ 0.3734,  0.1248,  0.0589,  1.0352,  0.7711,  1.7330, -0.6463,  0.4915],
         [-0.0800, -0.7955,  0.1118,  2.4803, -1.4088,  0.0790,  0.0483, -0.1531],
         [-1.1621,  0.5091, -0.9155, -0.0115, -0.3363,  0.5469, -1.3552,  0.1501],
         [-1.5694,  0.4089, -1.0754,  0.3191, -0.7785,  0.7567,  1.3531, -0.2555],
         [-0.1915, -0.0693, -0.5011, -1.2231, -0.9570, -0.9741,  0.8951, -2.2522],
         [ 1.5183,  0.7539,  0.4763, -1.8063,  1.9374,  1.5906,  1.3596,  0.2508]])]

In [67]:
import torch._tensor as tensor
torch_list = [torch.tensor(50., device='cuda:0'), torch.tensor(33.3333, device='cuda:0'), torch.tensor(50., device='cpu')]
torch_list2 = [torch.tensor(52., device='cuda:0'), torch.tensor(63.3333, device='cpu'), torch.tensor(80., device='cuda:0')]
n = len(torch_list)
new_list =[torch.empty(()) for _ in range(n)]
for i in range(len(torch_list)):
    # print(new_list[i])
    new_list[i] = torch_list[i].item()
    new_list[i] += torch_list2[i].item()
print(new_list)

[102.0, 96.6666030883789, 130.0]


In [68]:
torch.tensor(50., device='cuda:0')

tensor(50., device='cuda:0')

In [69]:
print(len(torch_list))

3


In [70]:
torch_list[0].item()

50.0

In [71]:
import torch
import torch.utils.data as data

class ClassProbabilitySampler(data.Sampler):
    def __init__(self, dataset, class_probabilities):
        self.dataset = dataset
        self.class_probabilities = class_probabilities

    def __iter__(self):
        indices = []
        num_samples = len(self.dataset)

        # Generate indices based on class probabilities
        for class_idx, class_prob in enumerate(self.class_probabilities):
            class_indices = [idx for idx in range(num_samples) if self.dataset[idx][1] == class_idx]
            num_class_samples = int(class_prob * num_samples)
            sampled_indices = torch.randperm(len(class_indices))[:num_class_samples]
            indices.extend([class_indices[idx] for idx in sampled_indices])

        return iter(indices)

    def __len__(self):
        return len(self.dataset)
    

In [72]:
x1

NameError: name 'x1' is not defined

In [74]:
import json

# Open the text file in read mode
with open('convert.txt', 'r') as file:
    # Read the contents of the file
    content = file.read()

    # Parse the content as a dictionary
    dictionary = json.loads(content)

# Print the dictionary
print(dictionary)

{'1': [[80.68039927328786, 91.62838056010585, 55.532934176537296, 58.24015367569462, nan, 67.629015744117, 81.76246022870464, 78.72601660451582, 91.94394142396989, 92.04803843344412], 9984], '2': [[82.77000491234564, 92.81375968686996, 62.5094970210906, 56.31542250110257, nan, 69.72877593501921, 79.70941838910502, 80.19354910081432, 90.55758947557018, 92.20395229708764], 9984], '3': [[85.47048886206842, 94.1914304671749, 55.405717529789094, 56.01712397913779, nan, 72.95834370274697, 82.81795324510144, 82.03712286180065, 91.24038514168032, 89.74437976960213], 9984], '4': [[86.1824704077936, 94.06695994715537, 61.008938450967115, 60.56403772907873, nan, 66.49265892274919, 82.74841151083669, 81.48927110241306, 91.20198260891821, 91.95203219998267], 9984], '5': [[86.93006168488533, 94.70963080621534, 60.73267341121551, 59.736616048505226, nan, 69.07649277717836, 84.90835017542685, 81.32081224995275, 91.61468102239793, 89.33142535301947], 9984], '6': [[86.04834104968656, 95.98433641003024, 

In [84]:
dictionary['1'][0][4] = 0

In [85]:
dictionary['1'][0]

[80.68039927328786,
 91.62838056010585,
 55.532934176537296,
 58.24015367569462,
 0,
 67.629015744117,
 81.76246022870464,
 78.72601660451582,
 91.94394142396989,
 92.04803843344412]